In [1]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
import torch

device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Lin

In [3]:
from datasets import load_dataset

data = load_dataset("glue", 'rte')
train_data = data['train']
train_data = {i:([t1,t2],l) for i, (t1, t2, l) in enumerate(zip(train_data['sentence1'], train_data['sentence2'], train_data['label']))}


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

import torch.nn as nn

crossentropy = nn.CrossEntropyLoss(reduce=False)

/home/donggunseo/.local/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [37]:
from tqdm import tqdm
id2label = {0:'yes', 1:'no'}
li=[]
for item in tqdm(list(train_data.items())[:100]):
    idx, ([t1,t2], l) = item
    sentence = t1 + " Therefore, " + t2
    inputs = tokenizer(sentence, return_tensors='pt')
    inputs_ids = inputs['input_ids']
    t1_ids = tokenizer.encode(t1, add_special_tokens=False)
    t2_ids = tokenizer.encode(t2, add_special_tokens=False)
    mid_ids = [7857, 29892]
    t1_start_idx = 1
    t1_end_idx = t1_start_idx+len(t1_ids)-1
    mid_start_idx = t1_end_idx+1
    mid_end_idx = mid_start_idx+1
    t2_start_idx = mid_end_idx+1
    t2_end_idx = t2_start_idx+len(t2_ids)-1
    inputs = inputs.to(device=model.device)
    with torch.no_grad():
        outputs = model.forward(**inputs, labels = inputs['input_ids'], return_dict=True)
    loss = crossentropy(outputs.logits[:,:-1,:].view(-1, outputs.logits[:,:-1,:].size(-1)), inputs['input_ids'][:,1:].view(-1)).detach().cpu()
    for i, item in enumerate(loss.tolist()):
        print("input: ", tokenizer.convert_ids_to_tokens(inputs_ids[0][i].item()), inputs_ids[0][i].item())
        print("gt_label: ", tokenizer.convert_ids_to_tokens(inputs_ids[0][i+1].item()), inputs_ids[0][i+1].item())
        print("perplexity: ",torch.exp(torch.tensor(item)).item())
        if torch.exp(torch.tensor(item)).item()>15:
            print("의심가는 인덱스: ",i)
        print("____________________")
        # li.append({'input' : tokenizer.convert_ids_to_tokens(inputs_ids[0][i]), 'gt_label': tokenizer.convert_ids_to_tokens(inputs_ids[0][i+1]), 'perplexity' : torch.exp(item).item()})
    break
    

  0%|          | 0/100 [00:00<?, ?it/s]

input:  <s> 1
gt_label:  ▁No 1939
perplexity:  778.546875
의심가는 인덱스:  0
____________________
input:  ▁No 1939
gt_label:  ▁We 1334
perplexity:  3719.409423828125
의심가는 인덱스:  1
____________________
input:  ▁We 1334
gt_label:  ap 481
perplexity:  2.140596389770508
____________________
input:  ap 481
gt_label:  ons 787
perplexity:  2.6946609020233154
____________________
input:  ons 787
gt_label:  ▁of 310
perplexity:  7.962088108062744
____________________
input:  ▁of 310
gt_label:  ▁Mass 7360
perplexity:  1.6587637662887573
____________________
input:  ▁Mass 7360
gt_label:  ▁Dest 15435
perplexity:  1.1458630561828613
____________________
input:  ▁Dest 15435
gt_label:  ruction 4080
perplexity:  1.0032269954681396
____________________
input:  ruction 4080
gt_label:  ▁Found 7460
perplexity:  4.337960243225098
____________________
input:  ▁Found 7460
gt_label:  ▁in 297
perplexity:  1.661431908607483
____________________
input:  ▁in 297
gt_label:  ▁Ira 21375
perplexity:  1.1385523080825806
_____

In [38]:
_, top_k_indices = torch.sort(outputs.logits[0][23].detach().cpu(), descending=True)

In [39]:
top_k_indices[:30]

tensor([19928,  4683,   399,  6975, 12074,  1222,  1938,   526,  7460,  2811,
         7440,  5240,  2610,   297,   341,  3872,  2216,   313,  1815,  1818,
         6527, 15808,   338,   892,   379, 21606,  5538,   505, 11662,  7835])

In [56]:
tokenizer.convert_ids_to_tokens(3872)

'▁Don'

In [50]:
sentence

'No Weapons of Mass Destruction Found in Iraq Yet. Therefore, Weapons of Mass Destruction Found in Iraq.'

In [24]:
templates = ["A %s one . ", "It was %s . ",
                     "All in all %s . ", "A %s piece . "]

label_words = ["terrible", "great"]

templates = [templates[0] % label_word for label_word in label_words]

In [25]:
from datasets import load_dataset

sst2 = load_dataset('sst2', split='train')

In [26]:
sst2_test=load_dataset('sst2', split='validation')

In [27]:
sst2_test

Dataset({
    features: ['idx', 'sentence', 'label'],
    num_rows: 872
})

In [37]:
templates = [template.strip() for template in templates]

In [39]:
templates

['A terrible one .', 'A great one .']

In [43]:
test_data = [(" "+sent, label) for sent, label in zip(sst2_test['sentence'], sst2_test['label'])]

In [44]:
test_data

[(" it 's a charming and often affecting journey . ", 1),
 (' unflinchingly bleak and desperate ', 0),
 (' allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker . ',
  1),
 (" the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales . ",
  1),
 (" it 's slow -- very , very slow . ", 0),
 (' although laced with humor and a few fanciful touches , the film is a refreshingly serious look at young women . ',
  1),
 (' a sometimes tedious film . ', 0),
 (" or doing last year 's taxes with your ex-wife . ", 0),
 (" you do n't have to know about music to appreciate the film 's easygoing blend of comedy and romance . ",
  1),
 (" in exactly 89 minutes , most of which passed as slowly as if i 'd been sitting naked on an igloo , formula 51 sank from quirky to jerky to utter turkey . ",
  0),
 (' the mesmerizing performances of the leads keep the film grounded and keep the audience riveted 

In [45]:
prefixes_with_space = [tokenizer(" "+template)["input_ids"] for template in templates]

In [46]:
prefixes_with_space

[[317, 7818, 530, 764], [317, 1049, 530, 764]]

In [47]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('gpt2-large')

In [49]:
test_inputs = [tokenizer(sent)["input_ids"] for sent, _ in test_data]

In [50]:
prefixes = [tokenizer(template)["input_ids"] for template in templates]

In [51]:
prefixes

[[32, 7818, 530, 764], [32, 1049, 530, 764]]

In [52]:
import numpy as np
idx = [idx for idx, _prefixes in enumerate(zip(*prefixes)) if not np.all([_prefixes[0]==_prefix for _prefix in _prefixes])][0]

In [36]:
templates

['A terrible one . ', 'A great one . ']

In [56]:
train_data = [(" "+sent, label) for sent, label in zip(sst2['sentence'][:4], sst2['label'][:4])]

In [57]:
demonstrations = []

np.random.shuffle(train_data)

for sent, label in train_data:
    if len(demonstrations)>0:
        prefixes = prefixes_with_space

    tokens = tokenizer(sent)["input_ids"]
    prefix = prefixes[(int(label))]

    tokens = prefix + tokens

    demonstrations += tokens

In [58]:
demonstrations

[317,
 7818,
 530,
 764,
 7808,
 649,
 3200,
 507,
 422,
 262,
 21694,
 4991,
 220,
 317,
 1049,
 530,
 764,
 326,
 10408,
 663,
 3435,
 290,
 48556,
 1223,
 2138,
 4950,
 546,
 1692,
 3450,
 220,
 317,
 7818,
 530,
 764,
 3793,
 15950,
 11378,
 284,
 3520,
 262,
 976,
 3690,
 220,
 317,
 7818,
 530,
 764,
 4909,
 645,
 20868,
 837,
 691,
 2248,
 1850,
 308,
 3775,
 220]

In [ ]:
input_tensors = []

for i in range(2):
    prefix = prefixes[i].copy()
    prefix = demonstrations.copy() + prefix
    tensor = prepro_sentence_pair([prefix], test_inputs, max_length,bos_token_id, eos_token_id, allow_truncation=use_demonstrations)